In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
image_size = (224, 224)  # Image resolution
batch_size = 32        # Batch size for training and validation

# Initialize ImageDataGenerator
data_gen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    validation_split=0.2  # Reserve 20% of data for validation
)

# Define training data generator
train_generator = data_gen.flow_from_directory(
    directory=r"C:\Users\riyat\OneDrive\Desktop\Satyanetra\deepf\test",  # Path to the train dataset directory
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'  # Use 80% of data for training
)

# Define validation data generator
val_generator = data_gen.flow_from_directory(
    directory=r"C:\Users\riyat\OneDrive\Desktop\Satyanetra\deepf\val",  # Path to the train dataset directory
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'  # Use 20% of data for validation
)

# Define test data generator (if available)
test_gen = ImageDataGenerator(rescale=1.0 / 255.0)

test_generator = test_gen.flow_from_directory(
    directory=r"C:\Users\riyat\OneDrive\Desktop\Satyanetra\deepf\train",  # Path to the test dataset directory
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)













Found 10806 images belonging to 2 classes.
Found 2518 images belonging to 4 classes.
Found 66722 images belonging to 4 classes.


In [ ]:

%pip install tensorflow
%pip install keras







Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Paths
train_dir = r"C:\Users\riyat\OneDrive\Desktop\Satyanetra\deepf\train"
val_dir = r"C:\Users\riyat\OneDrive\Desktop\Satyanetra\deepf\val"
test_dir = r"C:\Users\riyat\OneDrive\Desktop\Satyanetra\deepf\test"

# Data Generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary'
)
val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary'
)
test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary',
    shuffle=False  # Important for evaluation
)

# Model Setup
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, verbose=1)


#Training
history = model.fit(
    train_generator,
    validation_data=val_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator),
    epochs=5,
    callbacks=[checkpoint],
    verbose=1
)

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")


# Save Final Model
model.save(r"C:\Users\\riyat\\OneDrive\\Desktop\\Satyanetra\\my_model1.keras")
print("Model saved at the specified location.")










Found 66722 images belonging to 4 classes.
Found 12592 images belonging to 4 classes.
Found 13507 images belonging to 2 classes.


c:\Users\riyat\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
4171/4171 ━━━━━━━━━━━━━━━━━━━━ 0s 454ms/step - accuracy: 0.7052 - loss: 0.6290

c:\Users\riyat\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_loss improved from inf to 0.56526, saving model to best_model.keras
4171/4171 ━━━━━━━━━━━━━━━━━━━━ 2282s 546ms/step - accuracy: 0.7052 - loss: 0.6290 - val_accuracy: 0.7090 - val_loss: 0.5653
Epoch 2/5
4171/4171 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step - accuracy: 0.7004 - loss: 0.5863
Epoch 2: val_loss improved from 0.56526 to 0.54801, saving model to best_model.keras
4171/4171 ━━━━━━━━━━━━━━━━━━━━ 1035s 248ms/step - accuracy: 0.7004 - loss: 0.5863 - val_accuracy: 0.7004 - val_loss: 0.5480
Epoch 3/5
4171/4171 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.7096 - loss: 0.5693
Epoch 3: val_loss improved from 0.54801 to 0.54128, saving model to best_model.keras
4171/4171 ━━━━━━━━━━━━━━━━━━━━ 501s 120ms/step - accuracy: 0.7096 - loss: 0.5693 - val_accuracy: 0.6950 - val_loss: 0.5413
Epoch 4/5
4171/4171 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.7227 - loss: 0.5595
Epoch 4: val_loss improved from 0.54128 to 0.53856, saving model to best_model.keras
4171/4171 ━━━━━━━━━━━━━━━